<a href="https://colab.research.google.com/github/LeeGitaek/Kaggle_Solving/blob/master/SeoulGrandPark_Prediction_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Would remove:
    /usr/local/bin/kaggle
    /usr/local/lib/python3.6/dist-packages/kaggle-1.5.6.dist-info/*
    /usr/local/lib/python3.6/dist-packages/kaggle/*
Proceed (y/n)? y
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 4.5MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 2.2 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=f7941517f069e9a2c97be34e08c8d64588e4c4bb4a4e38fe03bff1ca933e97c6
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 68 Jun 19 09:57 kaggle.json


In [8]:
!kaggle competitions download -c 2020-ai-termproject-18011817

  0% 0.00/1.14k [00:00<?, ?B/s]
100% 1.14k/1.14k [00:00<00:00, 1.99MB/s]
  0% 0.00/466 [00:00<?, ?B/s]
100% 466/466 [00:00<00:00, 390kB/s]
  0% 0.00/24.3k [00:00<?, ?B/s]
100% 24.3k/24.3k [00:00<00:00, 19.6MB/s]


In [32]:
import pandas as pd
import numpy as np
import random

import torch
import torch.optim as optim
import torchvision.datasets as data
import torchvision.transforms as transforms

from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

In [57]:
device = torch.device('cuda')

torch.manual_seed(777)
random.seed(777)
torch.cuda.manual_seed_all(777)

learning_rate = 0.1
training_epochs = 8000
batch_size = 200
drop_prob = 0.3

In [58]:
xy_train = pd.read_csv('train_seoul_grandpark.csv', header = None, skiprows=1, usecols=range(1, 8))

x_data = xy_train.loc[: , 1:6]
y_data = xy_train.loc[: , [7]]

x_data = np.array(x_data)
y_data = np.array(y_data)

scaler = MinMaxScaler()
x_data = scaler.fit_transform(x_data)

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [59]:
train_dataset = TensorDataset(x_train, y_train)
data_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size, 
                                           shuffle = True, 
                                           drop_last = True)

In [60]:
linear1 = torch.nn.Linear(6, 4,bias=True)
linear2 = torch.nn.Linear(4, 4,bias=True)
linear3 = torch.nn.Linear(4, 1,bias=True)
#dropout = torch.nn.Dropout(p=drop_prob)
relu = torch.nn.SELU()

In [61]:
torch.nn.init.kaiming_normal_(linear1.weight)
torch.nn.init.kaiming_normal_(linear2.weight)
torch.nn.init.kaiming_normal_(linear3.weight)

model = torch.nn.Sequential(linear1,relu,
                            linear2,relu,
                            linear3).to(device)

In [62]:
loss = torch.nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

losses = []
model_history = []
err_history = []

total_batch = len(data_loader)

for epoch in range(training_epochs + 1):
  avg_cost = 0
  #model.train()
  
  for X, Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = loss(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
    
  model_history.append(model)
  err_history.append(avg_cost)
  
  if epoch % 100 == 0:  
    print('Epoch:', '%d' % (epoch + 1), 'Cost =', '{:.9f}'.format(avg_cost))
  losses.append(cost.item())
print('Learning finished')

Epoch: 1 Cost = 106923528.000000000
Epoch: 101 Cost = 51235000.000000000
Epoch: 201 Cost = 43704484.000000000
Epoch: 301 Cost = 41177308.000000000
Epoch: 401 Cost = 37934600.000000000
Epoch: 501 Cost = 40790416.000000000
Epoch: 601 Cost = 38731360.000000000
Epoch: 701 Cost = 40628800.000000000
Epoch: 801 Cost = 39127492.000000000
Epoch: 901 Cost = 39647432.000000000
Epoch: 1001 Cost = 38923752.000000000
Epoch: 1101 Cost = 40740036.000000000
Epoch: 1201 Cost = 39594408.000000000
Epoch: 1301 Cost = 37056172.000000000
Epoch: 1401 Cost = 37864448.000000000
Epoch: 1501 Cost = 40571464.000000000
Epoch: 1601 Cost = 39973944.000000000
Epoch: 1701 Cost = 40293052.000000000
Epoch: 1801 Cost = 40287916.000000000
Epoch: 1901 Cost = 37753632.000000000
Epoch: 2001 Cost = 38713832.000000000
Epoch: 2101 Cost = 39655280.000000000
Epoch: 2201 Cost = 39118472.000000000
Epoch: 2301 Cost = 39921032.000000000
Epoch: 2401 Cost = 38843060.000000000
Epoch: 2501 Cost = 38366184.000000000
Epoch: 2601 Cost = 3954

In [63]:
best_model = model_history[np.argmin(err_history)]

In [64]:
xy_test = pd.read_csv('test_seoul_grandpark.csv', header = None, skiprows=1, usecols = range(1, 7))
x_data = xy_test.loc[:, 1:6]
x_data = np.array(x_data)
x_data = scaler.transform(x_data)
x_test = torch.FloatTensor(x_data).to(device)

with torch.no_grad():
    model.eval()     
    predict = best_model(x_test)


In [65]:
submit = pd.read_csv('submit_sample.csv')
submit['Expected'] = submit['Expected'].astype(float)
for i in range(len(predict)):
  submit['Expected'][i] = predict[i]
submit.to_csv('submit.csv', mode = 'w', index = False, header = True)
submit

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Id,Expected
0,20170330,8388.352539
1,20180418,10198.683594
2,20190430,12329.935547
3,20180618,10470.603516
4,20160329,9991.739258
5,20170424,11660.111328
6,20180719,2527.253662
7,20170828,7479.009277
8,20180227,1390.254395
9,20160211,5202.645996


In [66]:
!kaggle competitions submit -c 2020-ai-termproject-18011817 -f submit.csv -m "14010974_이기택"

100% 767/767 [00:10<00:00, 74.4B/s]
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/kaggle/cli.py", line 64, in main
    print(out, end='')
UnicodeEncodeError: 'latin-1' codec can't encode characters in position 26-30: ordinal not in range(256)
